In [1]:
from keras import __version__
import numpy as np
from keras.models import model_from_json, Model, Sequential
from keras.layers import Dropout, Flatten, Dense, Activation
from keras.applications.inception_v3 import preprocess_input
from keras.preprocessing.image import img_to_array, load_img, ImageDataGenerator
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances
from keras import optimizers
from sklearn.model_selection import train_test_split
import keras

Using TensorFlow backend.


In [2]:
weights_path = 'inceptionv3_clothing_classifier.h5'
json_path = 'incep_filter_clothing_classifier.json'

json_file = open(json_path, 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

# load weights into new model
loaded_model.load_weights(weights_path)
print("Loaded model from disk")

# evaluate loaded model on test data
# loaded_model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
#score = loaded_model.evaluate(X, Y, verbose=0)
#print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))

Loaded model from disk


In [3]:
def image_preprocess(img_path):
    image = load_img(img_path, target_size=(299, 299))
    image = img_to_array(image)

    # our input image is now represented as a NumPy array of shape
    # (inputShape[0], inputShape[1], 3) however we need to expand the
    # dimension by making the shape (1, inputShape[0], inputShape[1], 3)
    # so we can pass it through thenetwork
    image = np.expand_dims(image, axis=0)

    # pre-process the image using the appropriate function based on the
    # model that has been loaded (i.e., mean subtraction, scaling, etc.)
    image = preprocess_input(image)
    return image

In [4]:
clothes_labels = ['dresses', 'jackets', 'jeans', 'shorts', 'skirts'  
                  'sweaters', 'sweatshirts', 'womens-outerwear',
                  'womens-pants', 'womens-tops']

def get_image_prediction(model, img_path):
    img = image_preprocess(img_path)
    preds = loaded_model.predict(img)[0]
    preds_labels = list(zip(clothes_labels, preds))
    preds_labels.sort(key=lambda p: p[1], reverse=True)
    return preds_labels

jeans_img_path = 'data/validation/jeans/666416190.jpg'
dress_img_path = 'data/validation/dresses/535993294.jpg'
jackets_img_path = 'data/validation/jackets/614521319.jpg'
jackets_img_path2 = 'data/validation/jackets/614763686.jpg'
get_image_prediction(loaded_model, jackets_img_path2)

[('jackets', 0.99018162),
 ('womens-outerwear', 0.0016008299),
 ('sweatshirts', 0.0011538396),
 ('womens-pants', 0.00047368027),
 ('shorts', 6.9756126e-05),
 ('dresses', 1.9559922e-05),
 ('jeans', 8.0368445e-06),
 ('skirtssweaters', 6.9709931e-06),
 ('womens-tops', 2.6730488e-06)]

In [5]:
loaded_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, None, None, 3) 0                                            
____________________________________________________________________________________________________
conv2d_1 (Conv2D)                (None, None, None, 32 864         input_1[0][0]                    
____________________________________________________________________________________________________
batch_normalization_1 (BatchNorm (None, None, None, 32 96          conv2d_1[0][0]                   
____________________________________________________________________________________________________
activation_1 (Activation)        (None, None, None, 32 0           batch_normalization_1[0][0]      
___________________________________________________________________________________________

In [6]:
loaded_model.layers.pop() # Get rid of the classification layer

In [7]:
last = loaded_model.layers[-1].output
model = Model(loaded_model.input, last)
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, None, None, 3) 0                                            
____________________________________________________________________________________________________
conv2d_1 (Conv2D)                (None, None, None, 32 864         input_1[0][0]                    
____________________________________________________________________________________________________
batch_normalization_1 (BatchNorm (None, None, None, 32 96          conv2d_1[0][0]                   
____________________________________________________________________________________________________
activation_1 (Activation)        (None, None, None, 32 0           batch_normalization_1[0][0]      
___________________________________________________________________________________________

In [8]:
dog_img_path = 'data-pets/train/dogs/dog.0.jpg'

img_jacket = image_preprocess(jackets_img_path)
img_jacket2 = image_preprocess(jackets_img_path2)
img_jeans = image_preprocess(jeans_img_path)
img_dress = image_preprocess(dress_img_path)
img_dog = image_preprocess(dog_img_path)

jacket_feat = model.predict(img_jacket)
jacket_feat2 = model.predict(img_jacket2)
jeans_feat = model.predict(img_jeans)
dress_feat = model.predict(img_dress)
dog_feat = model.predict(img_dog)

In [9]:
# compare cosine similarities
print('2 jackets: ', cosine_similarity(jacket_feat, jacket_feat2))
print('jeans and dress: ', cosine_similarity(jeans_feat, dress_feat))
print('jeans and dog: ', cosine_similarity(jeans_feat, dog_feat))

2 jackets:  [[ 0.98380375]]
jeans and dress:  [[ 0.95913839]]
jeans and dog:  [[ 0.96659511]]


In [10]:
jacket_feat.shape

(1, 1024)

## Simple Pairing Model

In [11]:
def get_image_pairs(clothing_classifier, filename, dir_path, y_output):
    pairs = []
    print('Reading file ', filename)
    with open(filename, 'r') as f:
        for i, line in enumerate(f.readlines()[:200]):
            if i % 50 == 0:
                print('Processing pair ', i)
            try:
                l = line.split()
                pair_id = l[0]
                product1_id = l[2]
                product2_id = l[5]
                product1_path = dir_path + pair_id + '_' + product1_id + '.jpg' 
                product2_path = dir_path + pair_id + '_' + product2_id + '.jpg' 

                product1_img = image_preprocess(product1_path)
                product2_img = image_preprocess(product2_path)

                product1_feat = clothing_classifier.predict(product1_img)
                product2_feat = clothing_classifier.predict(product2_img)

                merged_feat = np.concatenate((product1_feat, product2_feat), axis=1)
                # pairs.append((product1_feat, product2_feat, y_output))
                pairs.append((merged_feat, y_output))
            except Exception as e:
                print(e)

    return pairs



In [12]:
def get_data(clothing_classifier):
    valid_pairs = get_image_pairs(clothing_classifier, 
                                  'data-pairs/small_valid_clothing_pairs.tsv', 'data-pairs/valid/', 1)
    #invalid_pairs = get_image_pairs('data-pairs/small_invalid_clothing_pairs.tsv', 'data-pairs/invalid/', 0)
    invalid_pairs = get_image_pairs(clothing_classifier, 
                                    'data-pairs/small_unfashionable_clothing_pairs.tsv', 'data-pairs/unfashionable/', 0)
    # shuffle
    all_pairs_X = []
    all_pairs_y = []
    for i in range(len(valid_pairs)):
        all_pairs_X.append(valid_pairs[i][0])
        all_pairs_X.append(invalid_pairs[i][0])
        all_pairs_y.append(valid_pairs[i][1])
        all_pairs_y.append(invalid_pairs[i][1])
    return np.array(all_pairs_X), np.array(all_pairs_y)

#y_categorical = keras.utils.np_utils.to_categorical(y, num_classes=2)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

X, y = get_data(model)

Reading file  data-pairs/small_valid_clothing_pairs.tsv
Processing pair  0
Processing pair  50
Processing pair  100
Processing pair  150
Processing pair  200
Processing pair  250
Processing pair  300
Processing pair  350
Processing pair  400
Processing pair  450
Processing pair  500
Processing pair  550
Processing pair  600
Processing pair  650
Processing pair  700
Processing pair  750
Processing pair  800
Processing pair  850
Processing pair  900
Processing pair  950
Reading file  data-pairs/small_unfashionable_clothing_pairs.tsv
Processing pair  0
Processing pair  50
Processing pair  100
Processing pair  150
Processing pair  200
Processing pair  250
Processing pair  300
Processing pair  350
Processing pair  400
Processing pair  450
Processing pair  500
Processing pair  550
Processing pair  600
Processing pair  650
Processing pair  700
Processing pair  750
Processing pair  800
Processing pair  850
Processing pair  900
Processing pair  950


In [13]:
np.array(X).shape

(2000, 1, 2048)

In [14]:
def test_simple_network():
    '''Base network to be shared (eq. to feature extraction).
    '''
    seq = Sequential()
    seq.add(Dense(128, input_shape=(1,2048), activation='relu', name='fc1'))
    seq.add(Dropout(0.1))
    seq.add(Dense(64, activation='relu', name='fc2'))
    seq.add(Dropout(0.1))
    seq.add(Flatten())
    seq.add(Dense(1, activation='sigmoid', name='fc_final'))
    seq.compile(loss="binary_crossentropy", optimizer=optimizers.RMSprop(lr=0.00001), metrics=["accuracy"])
    return seq

In [19]:
fashion_classifier = test_simple_network()
fashion_classifier.summary()
fashion_classifier.fit(X, y, validation_split=0.5, epochs=50, batch_size=10)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
fc1 (Dense)                  (None, 1, 128)            262272    
_________________________________________________________________
dropout_3 (Dropout)          (None, 1, 128)            0         
_________________________________________________________________
fc2 (Dense)                  (None, 1, 64)             8256      
_________________________________________________________________
dropout_4 (Dropout)          (None, 1, 64)             0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 64)                0         
_________________________________________________________________
fc_final (Dense)             (None, 1)                 65        
Total params: 270,593
Trainable params: 270,593
Non-trainable params: 0
_________________________________________________________________
Trai

In [16]:
a = np.array([[1,2, 3]])
b = np.array([[4,5, 6]])
ab = np.concatenate((a, b), axis=1)
ab

array([[1, 2, 3, 4, 5, 6]])

In [17]:
ab.shape

(1, 6)

In [18]:
keras.utils.np_utils.to_categorical(np.array([1, 0, 0, 0]), num_classes=2)

array([[ 0.,  1.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.]])